In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

directory_path = "/Users/bao.vo/PycharmProjects/PythonProject/Data"

In [ ]:
# Set the directory containing your CSV files
directory_path = "/Users/bao.vo/PycharmProjects/PythonProject/Data/"

csv_files = glob.glob(os.path.join(directory_path, "*.csv"))

print(f"Found {len(csv_files)} CSV files.")

In [ ]:
eda_files = [
    "Georgia Milestones Scores.csv",
    "Georgia Milestones Sublevel Score.csv",
    "StudentTeacherGradeCombined",
    "GraduationAreaSummary.csv",
    "Courses.csv"
]

for fname in eda_files:
    fpath = os.path.join(directory_path, fname)
    try:
        try:
            df = pd.read_csv(fpath, encoding="utf-8", low_memory=False)
        except UnicodeDecodeError:
            df = pd.read_csv(fpath, encoding="latin1", low_memory=False)

        print(f"\n📊 First 5 rows of {fname}:")
        display(df.head(5))

    except FileNotFoundError:
        print(f"File not found: {fname}")
    except Exception as e:
        print(f"Failed to load {fname}: {e}")

In [72]:
# GraduationAreaSummary.csv
grad_summary = pd.read_csv(os.path.join(directory_path, "GraduationAreaSummary.csv"), low_memory=False)

# Courses.csv (static course catalog)
courses = pd.read_csv(os.path.join(directory_path, "Courses.csv"), low_memory=False)

all_grades = pd.read_csv(os.path.join(directory_path, "StudentTeacherGradeCombined.csv"), low_memory=False)
def categorize_grade_level(grade):
    if grade in ['K', '01', '02', '03', '04', '05']:
        return 'Elementary'
    elif grade in ['06', '07', '08']:
        return 'Middle'
    elif grade in ['09', '10', '11', '12']:
        return 'High'
    else:
        return 'Unknown'

all_grades['SchoolLevel'] = all_grades['GradeLevel'].apply(categorize_grade_level)

# Rename the “mask_studentpersonkey” column to student_id (if not already done)

if "mask_studentpersonkey" in all_grades.columns:
    df.rename(columns={"mask_studentpersonkey": "student_id"}, inplace=True)

# Inspect first few rows to verify
# print("=== GraduationAreaSummary Sample ===")
# display(grad_summary.head())
# print("\n=== Courses Sample ===")
# display(courses.head())
# print("\n=== Grades Sample ===")


In [73]:

# Filter only High School students with credit data
hs_grades = all_grades[
    (all_grades["SchoolLevel"] == "High") &
    (all_grades["AttemptedCredit"].notna()) &
    (all_grades["EarnedCredit"] > 0)
]
display(hs_grades.sort_values(by="AttemptedCredit", ascending=False).head(5))

,GradeLevel,SchoolYearId,SchoolYear,MarkingPeriodCode,MarkingPeriodStartDate,MarkingPeriodEndDate,SchoolDetailFCSId,SchoolStateId,CourseLevelDesc,CourseNumber,...,EndPeriodCode,SchoolCourseDesc,AttemptedCredit,EarnedCredit,MarkTypeCode,MarkTypeDesc,Mark,StaffPersonKey,mask_studentpersonkey,SchoolLevel
6048217,12,34,2023-2024,S1,2023-08-07 00:00:00.000,2023-12-15 00:00:00.000,753.0,1323.0,NaN,11.7116101,...,8th,WBL - Computing/Ntwk,1.5,1.5,S18,Semester,97,114329.0,584198.0,High
8288912,12,35,2024-2025,S1,2024-08-05 00:00:00.000,2024-12-20 00:00:00.000,753.0,1323.0,NaN,21.7116101,...,Advisement,WBL - Engineering,1.5,1.5,S18,Semester,70,114329.0,766803.0,High
6052782,12,34,2023-2024,S1,2023-08-07 00:00:00.000,2023-12-15 00:00:00.000,982.0,118.0,NaN,06.7116101,...,4,WBL - Entrepreneurship,1.5,1.5,S18,Semester,100,70017013.0,432989.0,High
6020559,12,34,2023-2024,S1,2023-08-07 00:00:00.000,2023-12-15 00:00:00.000,870.0,291.0,NaN,45.7115101,...,3,WBL - Social Studies,1.5,1.5,S18,Semester,94,24568.0,253993.0,High
8248668,12,35,2024-2025,S1,2024-08-05 00:00:00.000,2024-12-20 00:00:00.000,982.0,118.0,NaN,06.7116101,...,4,WBL - Entrepreneurship,1.5,1.5,S18,Semester,100,70017013.0,322508.0,High


In [74]:
# Make cleaned copies before modifying
hs_grades_cleaned = hs_grades.copy()
grad_summary_cleaned = grad_summary.copy()

In [75]:

# Normalize student ID format in both datasets
hs_grades_cleaned.loc[:, "student_id"] = hs_grades_cleaned["mask_studentpersonkey"].astype(str).str.replace(r"\.0$", "", regex=True)
grad_summary_cleaned["student_id"] = grad_summary_cleaned["mask_studentpersonkey"].astype(str).str.replace(r"\.0$", "", regex=True)
hs_grades_cleaned = hs_grades_cleaned.drop(columns=["mask_studentpersonkey"])
grad_summary_cleaned = grad_summary_cleaned.drop(columns=["mask_studentpersonkey"])

In [85]:
display(hs_grades_cleaned["student_id"].unique())
display(grad_summary_cleaned["student_id"].unique())

array(['370673', '311666', '329248', ..., '1988932', '1974938', '1988475'],
      shape=(48622,), dtype=object)

array(['350335', '1946451', '1987664', ..., '432012', '1996006',
       '1941168'], shape=(29316,), dtype=object)

In [86]:
display(hs_grades_cleaned[hs_grades_cleaned["student_id"] == "370673"])

,GradeLevel,SchoolYearId,SchoolYear,MarkingPeriodCode,MarkingPeriodStartDate,MarkingPeriodEndDate,SchoolDetailFCSId,SchoolStateId,CourseLevelDesc,CourseNumber,...,EndPeriodCode,SchoolCourseDesc,AttemptedCredit,EarnedCredit,MarkTypeCode,MarkTypeDesc,Mark,StaffPersonKey,SchoolLevel,student_id
3326453,09,33,2022-2023,S1,2022-08-08 00:00:00.000,2022-12-16 00:00:00.000,852.0,191.0,Level 2 Honors,23.0620041,...,1,10th Lit/Comp H,0.5,0.5,S18,Semester,100,7186.0,High,370673
3333184,09,33,2022-2023,S2,2023-01-04 00:00:00.000,2023-05-25 00:00:00.000,852.0,191.0,Level 2 Honors,23.0620042,...,1,10th Lit/Comp H,0.5,0.5,S18,Semester,103,113263.0,High,370673
3334147,09,33,2022-2023,S1,2022-08-08 00:00:00.000,2022-12-16 00:00:00.000,852.0,191.0,Level 2 Honors,26.0120041,...,5,Biology H,0.5,0.5,S18,Semester,94,6525.0,High,370673
3334789,09,33,2022-2023,S2,2023-01-04 00:00:00.000,2023-05-25 00:00:00.000,852.0,191.0,Level 2 Honors,26.0120042,...,5,Biology H,0.5,0.5,S18,Semester,94,6525.0,High,370673
3338890,09,33,2022-2023,S2,2023-01-04 00:00:00.000,2023-05-25 00:00:00.000,852.0,191.0,Level 2 Honors,60.0720042,...,6,Spanish 2 H,0.5,0.5,S18,Semester,100,73965.0,High,370673


In [87]:
display(grad_summary_cleaned[grad_summary_cleaned["student_id"] == "370673"])

,CurrentSchoolDetailFCSId,SchoolId,SchoolYearNumberFall,SubjectArea,SubjectAreaCreditRequired,AreaCredits,AreaCreditStillNeeded,student_id
11675,852,78,2022,Electives,4,3.0,1.0,370673
25729,852,78,2022,Health/ PersonalFitness,1,0.5,0.5,370673
29066,852,78,2022,Math,4,2.5,1.5,370673
50206,852,78,2022,World Language/ FineArts/ CareerTech,3,4.5,0.0,370673
180194,852,78,2022,ELA,4,2.0,2.0,370673
188024,852,78,2022,Science,4,2.5,1.5,370673
196600,852,78,2022,Social Studies,3,2.0,1.0,370673


In [67]:
display(hs_grades.head())

,GradeLevel,SchoolYearId,SchoolYear,MarkingPeriodCode,MarkingPeriodStartDate,MarkingPeriodEndDate,SchoolDetailFCSId,SchoolStateId,CourseLevelDesc,CourseNumber,...,EndPeriodCode,SchoolCourseDesc,AttemptedCredit,EarnedCredit,MarkTypeCode,MarkTypeDesc,Mark,StaffPersonKey,mask_studentpersonkey,SchoolLevel
3326453,09,33,2022-2023,S1,2022-08-08 00:00:00.000,2022-12-16 00:00:00.000,852.0,191.0,Level 2 Honors,23.0620041,...,1,10th Lit/Comp H,0.5,0.5,S18,Semester,100,7186.0,370673.0,High
3326455,09,33,2022-2023,S2,2023-01-04 00:00:00.000,2023-05-25 00:00:00.000,804.0,106.0,Level 2 Honors,40.0110042,...,6,Phys Sci H,0.5,0.5,S18,Semester,99,110809.0,311666.0,High
3326457,09,33,2022-2023,S1,2022-08-08 00:00:00.000,2022-12-16 00:00:00.000,741.0,910.0,Level 2 Honors,23.0620041,...,2,10th Lit/Comp H,0.5,0.5,S18,Semester,97,3931.0,329248.0,High
3326459,09,33,2022-2023,S2,2023-01-04 00:00:00.000,2023-05-25 00:00:00.000,804.0,106.0,Level 2 Honors,40.0110042,...,6,Phys Sci H,0.5,0.5,S18,Semester,98,110809.0,354631.0,High
3326461,09,33,2022-2023,S1,2022-08-08 00:00:00.000,2022-12-16 00:00:00.000,866.0,198.0,Level 2 Honors,23.0620041,...,1,10th Lit/Comp H,0.5,0.5,S18,Semester,101,111252.0,331279.0,High


In [80]:
display(grad_summary[grad_summary["mask_studentpersonkey"]=="584198"])

,mask_studentpersonkey,CurrentSchoolDetailFCSId,SchoolId,SchoolYearNumberFall,SubjectArea,SubjectAreaCreditRequired,AreaCredits,AreaCreditStillNeeded


In [81]:
print("GraduationAreaSummary SubjectArea:")
print(grad_summary["SubjectArea"].unique())

print("\nCourses DepartmentDesc:")
print(courses["DepartmentDesc"].unique())

print("\nCourses SubjectAreaDesc:")
print(courses["SubjectAreaDesc"].unique())

GraduationAreaSummary SubjectArea:
['Math' 'Health/ PersonalFitness' 'World Language/ FineArts/ CareerTech'
 'Social Studies' 'ELA' 'Science' 'Electives']

Courses DepartmentDesc:
['HEALTH EDUCATION' 'MATH' 'MISCELLANEOUS' nan
 'CAREER TECHNICAL AND AGRICULTURAL EDUCATION' 'FINE ARTS' 'LANGUAGE ARTS'
 'SCIENCE' 'PE/HEALTH' 'SPECIAL EDUCATION' 'READING' 'ELECTIVE COURSES'
 'SOCIAL SCIENCES' 'WORLD LANGUAGES' 'ESOL DEPT'
 'TALENTED AND GIFTED DEPT' 'ROTC DEPT' 'FOREIGN LANGUAGE'
 'CAREER TECHNOLOGY' 'FAMILY & CONSUMER SC' 'ART']

Courses SubjectAreaDesc:
['HEALTH' 'ELECTIVE' nan 'ENGLISH/LANGUAGE ARTS' 'SCIENCE'
 'CAREER/FOREIGN LANGUAGE/FINE ARTS' 'SR LANG ARTS C-P' 'FRENCH PATHWAY'
 'ENTREPRENEURSHIP PATHWAY' 'CAREER TECH ELECTIVE' 'JAPENESE PATHWAY'
 'PERSONAL FITNESS' 'SOCIAL STUDIES' 'MATH' 'ALGEBRA II OR EQUIVALENT'
 'MATH SR - C' 'BIOLOGY' 'PHYSICAL EDUCATION' 'SCIENCE C-P' 'ENGLISH C-P'
 'AM. LIT C-P' 'FINE ARTS' 'ANIMAL SYSTEMS PATHWAY'
 'AUDIO-VIDEO TECHNOLOGY & FILM PATHWAY'
 

In [ ]:
# Rename the student ID column in graduation summary if necessary
if "mask_studentpersonkey" in grad_summary.columns:
    grad_summary = grad_summary.rename(columns={"mask_studentpersonkey": "student_id"})

# Keep only the columns we need
grad_keep = ["student_id", "SubjectArea", "AreaCreditStillNeeded"]
grad_small = grad_summary[grad_keep].copy()

grad_pivot = grad_small.pivot_table(
    index="student_id",
    columns="SubjectArea",
    values="AreaCreditStillNeeded",
    aggfunc="last"
).fillna(0)

grad_pivot.columns = [f"CredStill_{col}" for col in grad_pivot.columns]
grad_pivot.reset_index(inplace=True)

print(">>> Pivoted GraduationAreaSummary (credits still needed) <<<")
display(grad_pivot.head())

In [ ]:

# Normalize GraduationAreaSummary SubjectArea for matching
subjectarea_map = {
    "Math": "MATH",
    "Health/ PersonalFitness": "HEALTH EDUCATION",
    "World Language/ FineArts/ CareerTech": "CAREER TECHNICAL AND AGRICULTURAL EDUCATION",
    "Social Studies": "SOCIAL SCIENCES",
    "ELA": "LANGUAGE ARTS",
    "Science": "SCIENCE",
    "Electives": "ELECTIVE COURSES"
}

grad_summary["NormalizedSubject"] = grad_summary["SubjectArea"].map(subjectarea_map)

# Use DepartmentDesc for mapping recommendations
credit_gaps = grad_summary[["student_id", "NormalizedSubject", "AreaCreditStillNeeded"]]
credit_gaps = credit_gaps[credit_gaps["AreaCreditStillNeeded"] > 0]
credit_gaps = credit_gaps.rename(columns={"NormalizedSubject": "DepartmentDesc"})

# Filter for active courses only
# active_courses = courses[courses["Active_status"] == "Active"]

# Cleaned course catalog for matching
course_map = courses[["CourseId", "coursename", "DepartmentDesc"]].dropna().drop_duplicates()

# Merge to get recommended courses by subject need
recommendations = credit_gaps.merge(course_map, on="DepartmentDesc", how="left")

In [ ]:
display(recommendations.head())

In [ ]:

recommendations = recommendations[["student_id", "DepartmentDesc", "SubjectArea", "AreaCreditStillNeeded", "CourseId", "coursename"]]

# Display recommendations
print("📘 Normalized Course Recommendations for Students with Credit Gaps")
display(recommendations.head())